In [1]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
with torch.profiler.profile() as profiler:
        pass
from sklearn.preprocessing import MinMaxScaler
import math

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def build_network(f1_layer_size, f2_layer_size, dropout, num_layers=2):


    network = nn.Sequential(  # fully-connected, dual hidden layer
        nn.Linear(128, f1_layer_size),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(f1_layer_size, f2_layer_size),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(f2_layer_size, 8),
        nn.Softmax(dim=1),
    )

    return network


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [106]:
class DogInput:
    def __init__(self, dogid, raceid,stats, dog, lstmCellh,lstmCellc) -> None:
        self.dogid= dogid
        self.raceid = raceid
        self.stats = stats
        self.dog = dog
        self.lstmCellh = lstmCellh.float()
        self.lstmCellc = lstmCellc.float()
        self.visited = 0
        
        
    def lstm_i(self, lstmInput):
        (self.lstmCellh,self.lstmCellc) = lstmInput
        self.visited = self.visited + 1
        # if self.visited>1:
        #     print("FOUND LEAK")
        #     sasdfasd

    def nextrace(self, raceid):
        self.nextrace_id = raceid

    def prevrace(self, raceid):
        self.prevrace_id = raceid

    def lstm_o(self, lstm_o):
        # print(lstm_o[0]._version)
        (lh,lc) = lstm_o
        if self.nextrace_id==-1:
            pass
        else:
            self.dog.races[self.nextrace_id].lstm_i((lh, lc))
            


In [168]:
class Dog:
    def __init__(self, dogid, hidden_size, layers) -> None:
        self.dogid = dogid
        # self.raceids = raceids #possible dictionary of race id keys dog stat outs
        self.lstmcell = 0
        self.layers = layers
        self.hidden_size = hidden_size
        self.l_debug = None
        self.races = {}

    def add_races(self, raceid, racedate, stats,nextraceid, prevraceid):
        self.races[raceid] = DogInput(self.dogid, raceid, stats, self, torch.randn(self.hidden_size).clone(),torch.randn(self.hidden_size).clone()) #this is the change
        self.races[raceid].nextrace(nextraceid)
        self.races[raceid].prevrace(prevraceid)

In [169]:
class Race:
    def __init__(self, raceid,trackOHE, dist, classes):
        self.raceid = raceid
        self.race_dist = dist
        self.race_track = trackOHE
        self.classes =  classes

    def add_dogs(self, dogs_list:DogInput):
        self.dog1 = dogs_list[0]
        self.dog2 = dogs_list[1]
        self.dog3 = dogs_list[2]
        self.dog4 = dogs_list[3]
        self.dog5 = dogs_list[4]
        self.dog6 = dogs_list[5]
        self.dog7 = dogs_list[6]
        self.dog8 = dogs_list[7]
        self.dogs = dogs_list

    def nn_input(self):
        input = torch.cat([x.stats for x in self.dogs], dim = 0)
        full_input = torch.cat((self.race_dist,self.race_track, input), dim=0)
        return full_input

    def lstm_input(self):
        
        l_input = [x.lstmCell for x in self.dogs]
        return l_input

    def list_dogs(self):
        dogs_l = [x for x in self.dogs]
        return dogs_l

    def pass_lstm_output(self, lstms):
        for i,dog in enumerate(self.dogs):
            lh,lc = lstms[i]
            lh,lc = lh.clone(), lc.clone()
            dog.lstm_o((lh,lc))
            dog.dog.l_debug = (lh,lc)
        # zipped_lstm = zip(self.dogs,lstms)
        # [x.lstm_o(y) for x,y in zipped_lstm]

In [170]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Net, self).__init__()
        self.lstm1 = nn.RNNCell(input_size, hidden_size)
        self.lstm2 = nn.RNNCell(input_size, hidden_size)
        self.lstm3 = nn.RNNCell(input_size, hidden_size)
        self.lstm4 = nn.RNNCell(input_size, hidden_size)
        self.lstm5 = nn.RNNCell(input_size, hidden_size)
        self.lstm6 = nn.RNNCell(input_size, hidden_size)
        self.lstm7 = nn.RNNCell(input_size, hidden_size)
        self.lstm8 = nn.RNNCell(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size * 8, 64)
        self.fc3 = nn.Linear(64, 8)
        self.hidden_size = hidden_size

    # x represents our data
    def forward(self, race: Race):
        x = race.nn_input().float()

        print(f"{x._version,race.dog1.lstmCellh._version, race.dog1.lstmCellc._version=}")

        (h1, c1) = self.lstm1(x, (race.dog1.lstmCellh, race.dog1.lstmCellc))
        # (h1, c1) = self.lstm1(x, (torch.rand(64), torch.rand(64)))
        (h2, c2) = self.lstm2(x, (race.dog2.lstmCellh, race.dog2.lstmCellc))
        (h3, c3) = self.lstm3(x, (race.dog3.lstmCellh, race.dog3.lstmCellc))
        (h4, c4) = self.lstm4(x, (race.dog4.lstmCellh, race.dog4.lstmCellc))
        (h5, c5) = self.lstm5(x, (race.dog5.lstmCellh, race.dog5.lstmCellc))
        (h6, c6) = self.lstm6(x, (race.dog6.lstmCellh, race.dog6.lstmCellc))
        (h7, c7) = self.lstm7(x, (race.dog7.lstmCellh, race.dog7.lstmCellc))
        (h8, c8) = self.lstm8(x, (race.dog8.lstmCellh, race.dog8.lstmCellc))

        lstm_list = [
            (h1, c1),
            (h2, c2),
            (h3, c3),
            (h4, c4),
            (h5, c5),
            (h6, c6),
            (h7, c7),
            (h8, c8)
        ]

        race.pass_lstm_output(lstm_list)
        # print(f"{h1=}\n{c1=}")
        # print(race.dog1.dog.races[race.dog1.nextrace_id].lstmCellh)
        # rejoin hidden states for rest of model
        xh = torch.cat((h1, h2, h3, h4, h5, h6, h7, h8), dim=0)
        xh = self.fc2(xh)
        x = self.fc3(xh)

        output = F.softmax(x, dim=0)
        return output


In [171]:
class Races:
    def __init__(self, hidden_size, layers) -> None:
        self.racesDict = {}
        self.dogsDict = {}
        self.raceIDs = []
        self.hidden_size = hidden_size
        self.layers = layers

    def add_race(self,raceid:str, trackOHE, dist, classes):
        self.racesDict[raceid] = Race(raceid, trackOHE, dist, classes)
        self.raceIDs.append(raceid)

    def add_dog(self,dogid):
        if dogid not in self.dogsDict.keys():
            self.dogsDict[dogid] = Dog(dogid, self.hidden_size, self.layers)
        else:
            self.dogsDict[dogid] = self.dogsDict[dogid]

    def get_race_input(self, idx) -> Race:
        raceidx = self.raceIDs[idx]
        # input = torch.cat([x.stats for x in self.racesDict[raceidx].dogs.values()], dim = 0)
        # full_input = torch.cat((self.racesDict[raceidx].race_dist,self.racesDict[raceidx].race_track, input), dim=0 )
        # dogs = [x for x in self.racesDict[raceidx].dogs]
        
        return self.racesDict[raceidx]

    def get_race_classes(self, idx):
        raceidx = self.raceIDs[idx]
        classes = [x for x in self.raceDict[raceidx].classes]
        return classes


In [172]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( 'dog_stats_df.npy', 'rb')
resultsdf = pickle.load(dog_stats_file)
dog_stats_df = pd.DataFrame(resultsdf)
dog_stats_df = dog_stats_df.fillna(-1).drop_duplicates(subset=['dogid', 'raceid'])
# dog_stats_df.to_excel('dog_stats_df.xlsx')
# dog_stats_df = dog_stats_df[dog_stats_df['dogid']=='674060006']
# dog_stats_df = dog_stats_df[dog_stats_df['raceid']!="130628253"]

In [173]:
print(dog_stats_df.size)
print(dog_stats_df.drop_duplicates(subset=['dogid', 'raceid']).size)

5270640
5270640


In [174]:
dog_stats_df[dog_stats_df['dogid']=="-159632"]

,dogid,raceid,date,trackOHE,dist,stats,place,startprice,margin,bfSP
2524,-159632,132951399,2016-07-11,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",450.0,"[6, 0.059524999999999995, -1, 0.05952499999999...",7.0,17.0,9.00,25.000000
8743,-159632,139325114,2016-08-02,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",400.0,"[8, 0.05930694444444444, 0.0003083771240174653...",6.0,11.2,6.70,18.766242
20697,-159632,151780470,2016-09-13,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",460.0,"[1, 0.05887962962962962, 0.0007715841355220415...",5.0,19.3,5.96,16.990319
44437,-159632,177083976,2016-12-05,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",450.0,"[3, 0.058605374396135265, 0.000835319370773274...",8.0,19.0,10.10,37.180216
71516,-159632,1967920009,2016-01-26,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",400.0,"[8, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",5.0,3.8,14.33,-1.000000


In [175]:
dog_stats_group = dog_stats_df.groupby(["dogid"])
layers = 1
hidden_size = 5
raceDB = Races(hidden_size,1)
print(raceDB)
for i,j in tqdm(dog_stats_group):
    #print(f"dog id = {i}\nlist = {j}")
    #tqdm.write(f"{i}")

    #creates next race column
    j = j.sort_values(['date'])
    j["next_race"] = j["raceid"].shift(-1).fillna(-1)
    j["prev_race"] = j["raceid"].shift(1).fillna(-1)
    raceDB.add_dog(i)

    
    j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),x['next_race'], x['prev_race']), axis=1)
    



100%|██████████| 25187/25187 [01:21<00:00, 310.77it/s]


In [176]:
softmin = nn.Softmin()

races_group = dog_stats_df.groupby(['raceid'])
null_dog = Dog("nullDog", raceDB.hidden_size, raceDB.layers)
null_dog_i = DogInput("nullDog", "-1", torch.zeros(16), null_dog, torch.zeros(raceDB.hidden_size), torch.zeros(raceDB.hidden_size))
null_dog_i.nextrace(-1)
null_dog_i.prevrace(-1)
for i,j in tqdm(races_group):
   dist = torch.tensor([j.dist.iloc[0]])
   trackOHE = torch.tensor(j.trackOHE.iloc[0])
   margin_list = [x for x in j["place"]]
   adjustedMargin = margin_list + ([8] * (8 - len(margin_list)))
   adjustedMargin = softmin(torch.tensor(adjustedMargin))
   raceDB.add_race(i,trackOHE,dist, adjustedMargin)

   
   dog_list = [raceDB.dogsDict[x].races[i] for x in j["dogid"]] # List of Dog Input??
   adjustedList = dog_list + ([null_dog_i] * (8 - len(dog_list)))
   raceDB.racesDict[i].add_dogs(adjustedList)

 # need to add in
    #j.apply(lambda x: raceDB.racesDict[i].add_dog(x['dogid'], raceDB.dogsDict[x['dogid']].races[i]), axis=1)

  0%|          | 0/72073 [00:00<?, ?it/s]C:\Users\Nick\AppData\Local\Temp\ipykernel_16608\1099167950.py:13: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  adjustedMargin = softmin(torch.tensor(adjustedMargin))
100%|██████████| 72073/72073 [00:21<00:00, 3322.70it/s]


In [177]:
newnet = Net(144,5)
#x = newnet.forward(raceDB.racesDict["132951399"])
#print(x)

In [178]:
loss_function = nn.MSELoss()
optimizer = optim.Adam(newnet.parameters(), lr=0.001)

In [179]:
newnet.forward

<bound method Net.forward of Net(
  (lstm1): LSTMCell(144, 5)
  (lstm2): LSTMCell(144, 5)
  (lstm3): LSTMCell(144, 5)
  (lstm4): LSTMCell(144, 5)
  (lstm5): LSTMCell(144, 5)
  (lstm6): LSTMCell(144, 5)
  (lstm7): LSTMCell(144, 5)
  (lstm8): LSTMCell(144, 5)
  (fc2): Linear(in_features=40, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
)>

In [183]:
torch.autograd.set_detect_anomaly(True)
list_t = [] 
for epoch in range(3): 
    for i in range(1000):  
        print(f"{i=}")
        print(f"{raceDB.get_race_input(i).raceid}")
        print(raceDB.racesDict['130628253'].dog1.lstmCellh)
        # raceDB.racesDict['130628253'].dog1.lstmCellh = torch.rand(5)
        # raceDB.racesDict['130628253'].dog1.lstmCellc = torch.rand(5)
        race = raceDB.get_race_input(i)
        X = race
        y = race.classes
        #print(y)
        output = newnet(X)
        #print(output) 
        #print([x.dogid for x in X.dogs])
        loss = loss_function(output, y)
        optimizer.zero_grad()
        newnet.zero_grad()
        loss.backward(retain_graph=True)  
        optimizer.step() 
    print(loss)

i=0
12539914
tensor([ 0.0000e+00, -1.1075e-18,  4.2698e-16,  0.0000e+00, -2.4178e-39],
       grad_fn=<CloneBackward0>)
x._version,race.dog1.lstmCellh._version, race.dog1.lstmCellc._version=(0, 0, 0)
i=1
126474686
tensor([0.8617, 0.5939, 0.8596, 0.5449, 0.6161])
x._version,race.dog1.lstmCellh._version, race.dog1.lstmCellc._version=(0, 0, 0)
i=2
128401375
tensor([0.0896, 0.0763, 0.2798, 0.0326, 0.6196])
x._version,race.dog1.lstmCellh._version, race.dog1.lstmCellc._version=(0, 0, 0)
i=3
128401378
tensor([0.3249, 0.2361, 0.0629, 0.2552, 0.2130])
x._version,race.dog1.lstmCellh._version, race.dog1.lstmCellc._version=(0, 0, 0)
i=4
128401379
tensor([0.5633, 0.4586, 0.1969, 0.9706, 0.1946])
x._version,race.dog1.lstmCellh._version, race.dog1.lstmCellc._version=(0, 0, 0)
i=5
128401380
tensor([0.8003, 0.0576, 0.5466, 0.3891, 0.2944])
x._version,race.dog1.lstmCellh._version, race.dog1.lstmCellc._version=(0, 0, 0)
i=6
128640871
tensor([0.7929, 0.0367, 0.9782, 0.1854, 0.4903])
x._version,race.dog1.l

c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "c:\Users\Nick\.conda\envs\pytorch\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Nick\.conda\envs\pytorch\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\traitlets\config\application.py", line 972, in launch_instance
    app.start()
  File "c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "c:\Users\Nick\.conda

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [5, 20]], which is output 0 of AsStridedBackward0, is at version 661; expected version 566 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [ ]:
raceDB.racesDict['130628253'].dog1.lstmCellh

tensor([ 5.4936e-15, -0.0000e+00, -1.8010e-01,  8.0139e-01,  0.0000e+00],
       grad_fn=<CloneBackward0>)

In [ ]:
x = torch.rand(10)
x._version
x = x +10
x.version_counter()

AttributeError: 'Tensor' object has no attribute 'version_counter'

In [ ]:
[x.raceid for x in raceDB.dogsDict['674060006'].races.values()]

['1927810002',
 '1965550008',
 '1974280023',
 '1984330020',
 '128662384',
 '130628253',
 '132426857',
 '134906777',
 '142979909',
 '153800009',
 '155979873',
 '158286451',
 '160032443',
 '161894160',
 '163739826',
 '164440238']

In [ ]:
j = dog_stats_group.get_group('674060006')
j["next_race"] = j["raceid"].shift(-1).fillna(-1)
j["prev_race"] = j["raceid"].shift(1).fillna(-1)
j

C:\Users\Nick\AppData\Local\Temp\ipykernel_23308\708876557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  j["next_race"] = j["raceid"].shift(-1).fillna(-1)
C:\Users\Nick\AppData\Local\Temp\ipykernel_23308\708876557.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  j["prev_race"] = j["raceid"].shift(1).fillna(-1)


,dogid,raceid,date,trackOHE,dist,stats,place,startprice,margin,bfSP,next_race,prev_race
81,674060006,128662384,2016-07-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[8, 0.05793068181818182, 0.0005795573379649154...",5.0,13.4,5.99,13.537126,130628253,-1
1247,674060006,130628253,2016-07-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[6, 0.05803062140391254, 0.000549412991266022,...",2.0,7.5,0.89,5.800000,132426857,128662384
2297,674060006,132426857,2016-07-09,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[2, 0.05790051783659378, 0.0005857007233663808...",5.0,7.2,12.41,7.332592,134906777,130628253
4362,674060006,134906777,2016-07-16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[4, 0.05811002794673681, 0.0007701509710601593...",6.0,10.0,5.49,8.292735,142979909,132426857
12043,674060006,142979909,2016-08-12,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[5, 0.05814002445339471, 0.0007180512688026204...",8.0,6.6,4.73,6.636301,153800009,134906777
22697,674060006,153800009,2016-09-20,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[8, 0.0581835379107531, 0.0006842431660542533,...",1.0,9.0,0.00,-1.000000,155979873,142979909
24670,674060006,155979873,2016-09-27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[1, 0.05806768411967779, 0.0007418818986516303...",1.0,5.3,0.00,5.700000,158286451,153800009
26970,674060006,158286451,2016-10-04,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",460.0,"[8, 0.057984258290616174, 0.000756246279258981...",4.0,16.0,2.51,19.042550,160032443,155979873
28511,674060006,160032443,2016-10-11,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",460.0,"[6, 0.057902236766398156, 0.000775014273555165...",5.0,3.2,5.71,4.400000,161894160,158286451
30161,674060006,161894160,2016-10-16,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",440.0,"[2, 0.05788801788470198, 0.0007437886077060855...",3.0,40.5,5.89,50.203159,163739826,160032443


In [ ]:
race = raceDB.get_race_input(i)
print(race)
[x.dogid for x in race.dogs]

['674060006',
 '992550020',
 '456300011',
 '531700011',
 '-630845',
 '422220005',
 '862520032',
 '471820000']

In [ ]:
dog_stats_df[dog_stats_df['dogid']=="674060006"]

,dogid,raceid,date,trackOHE,dist,stats,place,startprice,margin,bfSP
81,674060006,128662384,2016-07-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[8, 0.05793068181818182, 0.0005795573379649154...",5.0,13.4,5.99,13.537126
1247,674060006,130628253,2016-07-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[6, 0.05803062140391254, 0.000549412991266022,...",2.0,7.5,0.89,5.800000
2297,674060006,132426857,2016-07-09,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[2, 0.05790051783659378, 0.0005857007233663808...",5.0,7.2,12.41,7.332592
4362,674060006,134906777,2016-07-16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[4, 0.05811002794673681, 0.0007701509710601593...",6.0,10.0,5.49,8.292735
12043,674060006,142979909,2016-08-12,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[5, 0.05814002445339471, 0.0007180512688026204...",8.0,6.6,4.73,6.636301
22697,674060006,153800009,2016-09-20,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[8, 0.0581835379107531, 0.0006842431660542533,...",1.0,9.0,0.00,-1.000000
24670,674060006,155979873,2016-09-27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[1, 0.05806768411967779, 0.0007418818986516303...",1.0,5.3,0.00,5.700000
26970,674060006,158286451,2016-10-04,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",460.0,"[8, 0.057984258290616174, 0.000756246279258981...",4.0,16.0,2.51,19.042550
28511,674060006,160032443,2016-10-11,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",460.0,"[6, 0.057902236766398156, 0.000775014273555165...",5.0,3.2,5.71,4.400000
30161,674060006,161894160,2016-10-16,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",440.0,"[2, 0.05788801788470198, 0.0007437886077060855...",3.0,40.5,5.89,50.203159


In [ ]:
dog_stats_df[dog_stats_df['raceid']=="130628253"]

,dogid,raceid,date,trackOHE,dist,stats,place,startprice,margin,bfSP
1247,674060006,130628253,2016-07-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[6, 0.05803062140391254, 0.000549412991266022,...",2.0,7.5,0.89,5.800000
1248,992550020,130628253,2016-07-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[2, 0.05738770343580471, 0.003336316480057757,...",1.0,2.2,0.00,2.474907
1249,456300011,130628253,2016-07-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[3, 0.05791185897435897, 0.0008761459810322118...",3.0,5.2,4.93,7.124926
1250,531700011,130628253,2016-07-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[8, 0.057798295454545456, 0.000782510209493419...",8.0,7.5,25.69,11.500000
1251,-630845,130628253,2016-07-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[1, 0.06159493670886076, -1, 0.061594936708860...",7.0,26.5,14.61,-1.000000
1252,422220005,130628253,2016-07-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[5, 0.05926636500754148, 0.0014613987231697372...",6.0,30.4,12.07,-1.000000
1253,862520032,130628253,2016-07-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[4, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",5.0,6.6,10.67,-1.000000
1254,471820000,130628253,2016-07-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",400.0,"[7, 0.0584125, 0.0005126524163602494, 0.058775...",4.0,24.6,9.90,-1.000000


In [ ]:
race = raceDB.get_race_input(i)
[x.prevrace_id for x in race.dogs]

['128662384', '130216418', '129998982', -1, -1, -1, -1, -1]

In [ ]:
dog_stats_df[dog_stats_df['raceid']=='128662384']

,dogid,raceid,date,trackOHE,dist,stats,place,startprice,margin,bfSP
79,-749793,128662384,2016-07-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[7, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",7.0,4.0,10.56,3.575703
80,-722204,128662384,2016-07-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[5, 0.05739130434782609, 0.000540644787628083,...",6.0,25.2,7.21,-1.000000
81,674060006,128662384,2016-07-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[8, 0.05793068181818182, 0.0005795573379649154...",5.0,13.4,5.99,13.537126
82,641490415,128662384,2016-07-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[2, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",4.0,22.3,4.74,27.560705
83,855980015,128662384,2016-07-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[4, 0.058129268292682924, 0.001165988868575458...",2.0,4.0,1.21,4.500000
84,917310041,128662384,2016-07-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[3, 0.057680952380952374, 0.000871645249598771...",1.0,8.7,0.00,-1.000000
85,-747996,128662384,2016-07-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",395.0,"[6, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",3.0,2.4,2.94,-1.000000


In [ ]:
race = raceDB.get_race_input(70)
[x.dogid for x in race.dogs]

['992550020',
 '756890045',
 '176680008',
 '219770038',
 '641490147',
 '612290008',
 '851620023',
 'nullDog']

In [ ]:
dog_stats_df[dog_stats_df['raceid']=="131642760"]

,dogid,raceid,date,trackOHE,dist,stats,place,startprice,margin,bfSP
1506,219770038,131642760,2016-07-10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",350.0,"[8, 0.05658782290694055, 0.0012361330306929735...",2.0,5.3,1.46,6.599025
1507,881030036,131642760,2016-07-10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",350.0,"[7, 0.05652380952380952, 0.0025590531128656053...",1.0,5.9,0.00,6.288142
1508,714150041,131642760,2016-07-10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",350.0,"[3, 0.05705414339986733, 0.0014573440010096441...",3.0,2.3,1.83,2.463511
1509,70300070,131642760,2016-07-10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",350.0,"[4, 0.05630093167701864, 0.0009804515876057015...",4.0,4.2,4.43,4.217931
1510,32780008,131642760,2016-07-10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",350.0,"[1, 0.05782655128746308, 0.0014412990069647884...",5.0,26.3,5.79,-1.000000
1511,231490003,131642760,2016-07-10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",350.0,"[2, 0.05778867521367522, 0.001330971877428434,...",6.0,26.6,7.99,42.000000
1512,917310041,131642760,2016-07-10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",350.0,"[6, 0.05755869577030763, 0.0011394272834490246...",8.0,25.7,9.03,-1.000000
1513,641490147,131642760,2016-07-10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",350.0,"[5, 0.05471428571428571, -1, 0.054714285714285...",7.0,27.7,8.49,30.858631
